<a href="https://colab.research.google.com/github/sbrunswi/COVIDExposureIndices/blob/master/rill_safegraph_data_access_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

A Fast, Easy Way to Query SafeGraph Data in a Python Notebook
---

Welcome to Rill Data's tutorial Jupyter notebook for the [SafeGraph COVID-19 Consortium](https://www.safegraph.com/covid-19-data-consortium).  Our motivation is to show a simple, secure, alternative data access path to S3 downloads, enabling SQL-defined subsets of data to be pulled directly into a notebook.

Rill hosts a cloud database table that this code queries, which is a unified data set created by our talented colleages at [Unfolded](https://unfolded.ai), containing SafeGraph's population movement data, case data from The New York Times Covid Github repository, and spending data from Facteus.  It contains a row for every one of 220,000 census block groups in the US, for every day since January 1, 2020, resulting in 36 millions of rows as of this writing. [This schema describes the meaning of each of the 188 columns](https://docs.google.com/spreadsheets/d/1gxBKcSjIjwBsJswA3JvjZtqBLFBssus8wO-xpUeLWYw/edit#gid=0).  

Queries against this table should return almost instantly, but if you experience performance issues, please contact us [#rill-support](https://safegraphcovid19.slack.com/archives/C0139RQSTRP) and direct database schema questions to [#unfolded-studio-support](https://safegraphcovid19.slack.com/archives/C012T1Z8FQA).

In [ ]:
import json
import requests as rq
import ipywidgets as widgets
from IPython.display import display
import pandas as pd


### Get your access token and validate it
Visit https://covid.rilldata.com/accesstoken to copy your personal access token string (use the same email you use for the Consortium Slack channel), and paste it in below (then hit return ⏎) to validate this notebook's access to the database.

In [ ]:
text = widgets.Text(description="Enter token", width=400)
token = ''
display(text)
def handle_token(sender): 
  global token
  token = text.value
  print('Token updated Successfully!')
text.on_submit(handle_token)

Text(value='', description='Enter token')

`Token updated Successfully!` should appear as a message above, meaning this notebook now has validated access to the Rill cloud database.  

### Query Rill's cloud database

Next, let's set up the database URL and authentication headers.

In [ ]:
database_url = 'https://druid.ws.demo.rilldata.io/druid/v2/sql'
headers = {
                'Authorization': 'Bearer {}'.format(token)
          }

Next we'll issue a set of example queries, beginning with a simple query to pull just 5 rows of data into a simple pandas data frame.

In [ ]:
sql = """
SELECT FLOOR("__time" TO DAY) AS "__timestamp",
       AVG(1.0*completely_home_device_count/device_count) AS "AVG(1.0*completely_home_device_count/dev..."
FROM "druid"."covid_safegraph_facteus_nytimes"
WHERE "__time" >= '2020-04-19 00:00:00.000000'
  AND "__time" < '2020-06-18 00:00:00.000000'
  AND "device_count" != 0
GROUP BY FLOOR("__time" TO DAY)
ORDER BY "AVG(1.0*completely_home_device_count/dev..." DESC
LIMIT 10
"""
res = rq.post(database_url, headers=headers, json= {"query": sql })
print("Status Code:", res.status_code)  # Should be 200, otherwise, you may have a permissions issue
df = pd.read_json(json.dumps(res.json()))
df

Status Code: 200


,__timestamp,AVG(1.0*completely_home_device_count/dev...
0,2020-04-19T00:00:00.000Z,0.442363
1,2020-04-26T00:00:00.000Z,0.421956
2,2020-05-03T00:00:00.000Z,0.419265
3,2020-04-20T00:00:00.000Z,0.406592
4,2020-05-17T00:00:00.000Z,0.398933
5,2020-04-21T00:00:00.000Z,0.397235
6,2020-05-04T00:00:00.000Z,0.392129
7,2020-05-11T00:00:00.000Z,0.388055
8,2020-05-05T00:00:00.000Z,0.387569
9,2020-04-23T00:00:00.000Z,0.386580
